In [1]:
import os
import openpyxl
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.styles import PatternFill
import glob
import numpy as np

In [2]:
color=[['c6efce','006100'],
       ['ffeb9c','9c6500']]
c_lebal=0

In [3]:
def cal_heat(C31,D31,E31,F31,G31,H31,I31,sugar):#rice,meat,veg,fruit,milk,oil,seed,sugar
    protein=(C31*2)+(D31*7)+(E31*1)+(G31*8)
    fat=(D31*5)+(G31*8)+(H31*5)+(I31*5)
    carbo=(C31*15)+(E31*5)+(F31*15)+(G31*12)
    heat=(protein*4)+(fat*9)+(carbo*4)+(sugar*4)
    return heat

In [4]:
def area(string):
    ans=np.zeros((2,4),dtype=np.float64)
    string=string.split('.')
    if len(string)>2:
        string=string[0]+'.'+string[1]
    else:
        string=string[0]
    d1=0
    for i in glob.glob('./label/'+string+'*'):
        f=open(i,'r',encoding='UTF-8')
        content=f.read().split('\n')
        for value in content:
            text=value.split(' ')
            if text[0]=='':
                continue
            try:
                ans[d1][int(text[0])]+=float(text[3])*float(text[4])
            except:
                print(i)
        d1+=1
        f.close()
    return ans

In [5]:
print('程式啟動，初始化中...')
success=0
failed=0
error=[]
wb=openpyxl.Workbook()
sheet=wb.create_sheet("營養成分",0)
titles=('檔案名','全榖雜糧(份)','豆魚蛋肉(份)','蔬菜(份)','油脂(份)','熱量(大卡)','vegtable','rice','meat','fried meat')
sheet.append(titles)
name=os.listdir('KS/')
name_count=2
for ID in name:
    section=area(ID)
    #print(ID)
    source=load_workbook('KS/'+ID, data_only=True, read_only=True)
    if '統計總表' not in source.sheetnames:
        error.append(ID)
        failed+=1
        continue
    sheet10=source['統計總表']
    fille=PatternFill('solid', fgColor=color[c_lebal%2][0])
    sheet.cell(row=name_count,column=1,value=ID).fill=fille
    sheet.merge_cells(start_row=name_count, start_column=1, end_row=name_count+1, end_column=1)
    iner_leb=2
    start=0
    for k in range(1,60):
        if sheet10.cell(row=1,column=k).value=='全榖雜糧類':
            start=k
            break
    for i in range(8):
        if i==3 or i==4 or i==6:
            continue
        sheet.cell(row=name_count,column=iner_leb,value=sheet10.cell(row=3,column=i+start).value).fill=fille
        sheet.cell(row=name_count+1,column=iner_leb,value=sheet10.cell(row=4,column=i+start).value).fill=fille
        iner_leb+=1 
    for i in range(4):
        sheet.cell(row=name_count,column=iner_leb,value=section[0][i]).fill=fille
        sheet.cell(row=name_count+1,column=iner_leb,value=section[1][i]).fill=fille
        iner_leb+=1
    c_lebal+=1
    name_count+=2
    source.close()
    #print('營養資料表已轉換: '+ID)
    success+=1
wb.save('營養份數表_all new.xlsx')
wb.close()
print('資料提取結束,',success,'筆成功,',failed,'筆失敗。')
#sheet.merge_cells(start_row=2, start_column=1, end_row=3, end_column=1)

程式啟動，初始化中...
資料提取結束, 100 筆成功, 0 筆失敗。
